In [2]:
import pandas as pd

small_dataset = 'ml-latest-small'
large_dataset = 'ml-latest'

cleaned_small_dataset_dir = 'ml-latest-small-cleaned'

from enum import Enum
import os

class MovieLenseData(Enum):
    movie = 'movies.csv'
    links = 'links.csv'
    ratings = 'ratings.csv'
    tags = 'tags.csv'
    
    def read_csv(self, dataset_path):
        csv_path = os.path.join(dataset_path, self.value)
        return pd.read_csv(csv_path)
    def write_csv(self, dataset_path, df):
        file_name = os.path.join(dataset_path, self.value)
        df.to_csv(file_name)

In [4]:
import tmdbsimple as tmdb
tmdb.API_KEY = 'your_key_here'

In [6]:
    links_df = MovieLenseData.links.read_csv(small_dataset)
links_df

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [24]:
links_df[links_df.tmdbId.notnull()].count()

movieId    9734
imdbId     9734
tmdbId     9734
dtype: int64

In [41]:
def get_cast_df(movie):
    credits = movie.credits()
    cast_df_this_movie = pd.DataFrame(credits['cast'])
    cast_df_this_movie['tmdbId'] = row['tmdbId']
    cast_df_this_movie['movieId'] = row['movieId']
    return cast_df_this_movie

def get_movie_info_df(movie):
    movie_info = movie.info()
    movie_info_this_movie_df = pd.DataFrame([movie_info])
    movie_info_this_movie_df['tmdbId'] = row['tmdbId']
    movie_info_this_movie_df['movieId'] = row['movieId']
    return movie_info_this_movie_df

In [52]:
all_movie_info_df = []
all_cast_df = []

count = 0
for index, row in links_df[links_df.tmdbId.notnull()].iterrows():
#     print('movie id', row['movieId'], 'tmdb id', row['tmdbId'])
    try:
        movie = tmdb.Movies(row['tmdbId'])

        cast_df_this_movie = get_cast_df(movie)
        all_cast_df.append(cast_df_this_movie)

#         movie_info = movie.info()

#         movie_info_this_movie_df = pd.DataFrame([movie_info])
#         movie_info_this_movie_df['tmdbId'] = row['tmdbId']
#         movie_info_this_movie_df['movieId'] = row['movieId']
#         all_movie_info_df.append(movie_info_this_movie_df)
        count += 1
    except Exception as e:
        print('movie id', row['movieId'], 'tmdb id', row['tmdbId'])
        print(e)
    if count % 100 == 0:
        print('count', count, 'index', index)
        


count 100 index 99
count 200 index 199
count 300 index 299
count 400 index 399
count 500 index 499
count 600 index 599
count 700 index 700
count 800 index 800
count 900 index 901
count 1000 index 1001
count 1100 index 1101
count 1200 index 1201
count 1300 index 1301
count 1400 index 1401
count 1500 index 1501
count 1600 index 1601
count 1700 index 1701
count 1800 index 1801
count 1900 index 1901
count 2000 index 2001
count 2100 index 2101
count 2200 index 2202
count 2300 index 2302
count 2400 index 2402
count 2500 index 2502
count 2600 index 2602
count 2700 index 2702
count 2800 index 2802
count 2900 index 2902
count 3000 index 3002
count 3100 index 3103
movie id 4207.0 tmdb id 12773.0
404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/12773.0/credits?api_key=b2fdbc7a0dd12de57e6b7a8ba61a6eec
count 3200 index 3204
count 3300 index 3304
movie id 4568.0 tmdb id 17882.0
404 Client Error: Not Found for url: https://api.themoviedb.org/3/movie/17882.0/credits?api_key=b2fd

In [54]:
# movie_info_df = pd.concat(all_movie_info_df)
cast_df = pd.concat(all_cast_df)

In [55]:
cast_df

,adult,gender,id,known_for_department,name,original_name,popularity,profile_path,cast_id,character,credit_id,order,tmdbId,movieId
0,False,2.0,31.0,Acting,Tom Hanks,Tom Hanks,31.787,/xndWFsBlClOJFRdhSt4NBwiPq2o.jpg,14.0,Woody (voice),52fe4284c3a36847f8024f95,0.0,862.0,1.0
1,False,2.0,12898.0,Acting,Tim Allen,Tim Allen,6.733,/6qlDjidQSKNcJFHzTXh0gQS83ub.jpg,15.0,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,1.0,862.0,1.0
2,False,2.0,7167.0,Acting,Don Rickles,Don Rickles,2.322,/iJLQV4dcbTUgxlWJakjDldzlMXS.jpg,16.0,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2.0,862.0,1.0
3,False,2.0,12899.0,Acting,Jim Varney,Jim Varney,3.982,/yrhgPWGOc36GN0P9kuJ48sU4fVa.jpg,17.0,Slinky Dog (voice),52fe4284c3a36847f8024fa1,3.0,862.0,1.0
4,False,2.0,12900.0,Acting,Wallace Shawn,Wallace Shawn,5.282,/jviZU3Ae0vVKW6cYeEtjfxq2TWS.jpg,18.0,Rex (voice),52fe4284c3a36847f8024fa5,4.0,862.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,False,0.0,559446.0,Acting,Alex Thaler,Alex Thaler,0.600,None,1023.0,Background Actor (uncredited),52fe467b9251416c91055d9f,19.0,37891.0,193609.0
20,False,1.0,6479.0,Production,Sharon Bialy,Sharon Bialy,2.489,/cgz8PwlI1iJHFEZZ6BguMUv6U4E.jpg,1024.0,,52fe467b9251416c91055da3,20.0,37891.0,193609.0
21,False,1.0,25830.0,Production,Debi Manwiller,Debi Manwiller,1.214,/2LGUnaXYCwPMjqWsBtDVajFZMm7.jpg,1025.0,,52fe467b9251416c91055da7,21.0,37891.0,193609.0
22,False,1.0,60056.0,Production,Mary Margiotta,Mary Margiotta,1.400,None,1026.0,,52fe467b9251416c91055dab,22.0,37891.0,193609.0


In [56]:
# movie_info_csv = os.path.join(cleaned_small_dataset_dir, 'movie_info.csv')
# movie_info_df.to_csv(movie_info_csv, index=False)

cast_csv_path = os.path.join(cleaned_small_dataset_dir, 'cast.csv')
cast_df.to_csv(cast_csv_path, index=False)

In [38]:
cast_df

,adult,gender,id,known_for_department,name,original_name,popularity,profile_path,cast_id,character,credit_id,order
0,False,2,31,Acting,Tom Hanks,Tom Hanks,31.787,/xndWFsBlClOJFRdhSt4NBwiPq2o.jpg,14,Woody (voice),52fe4284c3a36847f8024f95,0
1,False,2,12898,Acting,Tim Allen,Tim Allen,6.733,/6qlDjidQSKNcJFHzTXh0gQS83ub.jpg,15,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,1
2,False,2,7167,Acting,Don Rickles,Don Rickles,2.322,/iJLQV4dcbTUgxlWJakjDldzlMXS.jpg,16,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2
3,False,2,12899,Acting,Jim Varney,Jim Varney,3.982,/yrhgPWGOc36GN0P9kuJ48sU4fVa.jpg,17,Slinky Dog (voice),52fe4284c3a36847f8024fa1,3
4,False,2,12900,Acting,Wallace Shawn,Wallace Shawn,5.282,/jviZU3Ae0vVKW6cYeEtjfxq2TWS.jpg,18,Rex (voice),52fe4284c3a36847f8024fa5,4
...,...,...,...,...,...,...,...,...,...,...,...,...
19,False,0,559446,Acting,Alex Thaler,Alex Thaler,0.600,None,1023,Background Actor (uncredited),52fe467b9251416c91055d9f,19
20,False,1,6479,Production,Sharon Bialy,Sharon Bialy,2.489,/cgz8PwlI1iJHFEZZ6BguMUv6U4E.jpg,1024,,52fe467b9251416c91055da3,20
21,False,1,25830,Production,Debi Manwiller,Debi Manwiller,1.214,/2LGUnaXYCwPMjqWsBtDVajFZMm7.jpg,1025,,52fe467b9251416c91055da7,21
22,False,1,60056,Production,Mary Margiotta,Mary Margiotta,1.400,None,1026,,52fe467b9251416c91055dab,22


In [39]:
all_cast_df[0]

,adult,gender,id,known_for_department,name,original_name,popularity,profile_path,cast_id,character,credit_id,order
0,False,2,31,Acting,Tom Hanks,Tom Hanks,31.787,/xndWFsBlClOJFRdhSt4NBwiPq2o.jpg,14,Woody (voice),52fe4284c3a36847f8024f95,0
1,False,2,12898,Acting,Tim Allen,Tim Allen,6.733,/6qlDjidQSKNcJFHzTXh0gQS83ub.jpg,15,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,1
2,False,2,7167,Acting,Don Rickles,Don Rickles,2.322,/iJLQV4dcbTUgxlWJakjDldzlMXS.jpg,16,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2
3,False,2,12899,Acting,Jim Varney,Jim Varney,3.982,/yrhgPWGOc36GN0P9kuJ48sU4fVa.jpg,17,Slinky Dog (voice),52fe4284c3a36847f8024fa1,3
4,False,2,12900,Acting,Wallace Shawn,Wallace Shawn,5.282,/jviZU3Ae0vVKW6cYeEtjfxq2TWS.jpg,18,Rex (voice),52fe4284c3a36847f8024fa5,4
5,False,2,7907,Acting,John Ratzenberger,John Ratzenberger,6.352,/oRtDEOuIO1yDhTz5dORBdxXuLMO.jpg,19,Hamm (voice),52fe4284c3a36847f8024fa9,5
6,False,1,8873,Acting,Annie Potts,Annie Potts,5.689,/hWIzkAH7jkSkxBfwdudxbSPxeno.jpg,20,Bo Peep (voice),52fe4284c3a36847f8024fad,6
7,False,2,1116442,Acting,John Morris,John Morris,0.682,/xMtT5UlPYZX7tobi55YFcdgyFp.jpg,26,Andy (voice),52fe4284c3a36847f8024fc1,7
8,False,2,12901,Acting,Erik von Detten,Erik von Detten,1.257,/7fHjKnLKmqzR0kmvRPLxpQE7BsK.jpg,22,Sid (voice),52fe4284c3a36847f8024fb1,8
9,False,1,12133,Acting,Laurie Metcalf,Laurie Metcalf,1.595,/paZsjbxC1yOw9U3zrrkzcElyaV3.jpg,23,Mrs. Davis (voice),52fe4284c3a36847f8024fb5,9


In [18]:
pd.DataFrame([movie_info])

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,/3Rfvhy1Nl6sSGJwyjb0QiZzZYlB.jpg,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 12, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,...,1995-10-30,373554033,81,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,Toy Story,False,7.9,13839


In [31]:
cast_df_movie_1['tmdbId'] = 5

In [32]:
cast_df_movie_1

,adult,gender,id,known_for_department,name,original_name,popularity,profile_path,cast_id,character,credit_id,order,tmdbId
0,False,2,31,Acting,Tom Hanks,Tom Hanks,31.787,/xndWFsBlClOJFRdhSt4NBwiPq2o.jpg,14,Woody (voice),52fe4284c3a36847f8024f95,0,5
1,False,2,12898,Acting,Tim Allen,Tim Allen,6.733,/6qlDjidQSKNcJFHzTXh0gQS83ub.jpg,15,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,1,5
2,False,2,7167,Acting,Don Rickles,Don Rickles,2.322,/iJLQV4dcbTUgxlWJakjDldzlMXS.jpg,16,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2,5
3,False,2,12899,Acting,Jim Varney,Jim Varney,3.982,/yrhgPWGOc36GN0P9kuJ48sU4fVa.jpg,17,Slinky Dog (voice),52fe4284c3a36847f8024fa1,3,5
4,False,2,12900,Acting,Wallace Shawn,Wallace Shawn,5.282,/jviZU3Ae0vVKW6cYeEtjfxq2TWS.jpg,18,Rex (voice),52fe4284c3a36847f8024fa5,4,5
5,False,2,7907,Acting,John Ratzenberger,John Ratzenberger,6.352,/oRtDEOuIO1yDhTz5dORBdxXuLMO.jpg,19,Hamm (voice),52fe4284c3a36847f8024fa9,5,5
6,False,1,8873,Acting,Annie Potts,Annie Potts,5.689,/hWIzkAH7jkSkxBfwdudxbSPxeno.jpg,20,Bo Peep (voice),52fe4284c3a36847f8024fad,6,5
7,False,2,1116442,Acting,John Morris,John Morris,0.682,/xMtT5UlPYZX7tobi55YFcdgyFp.jpg,26,Andy (voice),52fe4284c3a36847f8024fc1,7,5
8,False,2,12901,Acting,Erik von Detten,Erik von Detten,1.257,/7fHjKnLKmqzR0kmvRPLxpQE7BsK.jpg,22,Sid (voice),52fe4284c3a36847f8024fb1,8,5
9,False,1,12133,Acting,Laurie Metcalf,Laurie Metcalf,1.595,/paZsjbxC1yOw9U3zrrkzcElyaV3.jpg,23,Mrs. Davis (voice),52fe4284c3a36847f8024fb5,9,5
